### Colab_Version

In [ ]:
# 安装必要依赖（只需首次运行）
!pip install osmnx geopandas matplotlib

In [ ]:
import os
import osmnx as ox
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# 获取 Greater London 区域边界
place_name = "Greater London, United Kingdom"
gdf = ox.geocode_to_gdf(place_name)

# 自定义过滤器：仅保留允许骑行的道路
custom_filter = (
    '["highway"]["area"!~"yes"]'
    '["highway"!~"abandoned|bus_guideway|construction|corridor|elevator|escalator|footway|'
    'motor|no|planned|platform|proposed|raceway|razed|steps"]'
    '["bicycle"!~"no"]["service"!~"private"]'
)

# 下载骑行网络
print("正在抓取骑行网络数据（Greater London）")
graph = ox.graph_from_polygon(gdf.geometry[0], custom_filter=custom_filter, simplify=True)

# 转换为 GeoDataFrame（只保留边）
edges = ox.graph_to_gdfs(graph, nodes=False)


In [ ]:
# 构造 LTS 所需字段
edges['functional_class'] = edges['highway']
edges['has_bike_lane'] = edges['cycleway'].notnull() if 'cycleway' in edges.columns else False
edges['speed_limit'] = pd.to_numeric(edges['maxspeed'], errors='coerce').fillna(30)
edges['lane_count'] = pd.to_numeric(edges['lanes'], errors='coerce').fillna(1)

#定义简化版 LTS 判定规则
def assign_lts(row):
    fc = row['functional_class']
    bike = row['has_bike_lane']
    speed = row['speed_limit']
    lanes = row['lane_count']

    if isinstance(fc, list): fc = fc[0]
    if pd.isna(fc): fc = 'unclassified'

    if bike and speed <= 25 and lanes <= 2 and fc in ['residential', 'unclassified', 'living_street']:
        return 1
    if (not bike and speed <= 30 and lanes <= 2) or (bike and speed <= 35 and lanes <= 3):
        if fc in ['residential', 'tertiary', 'unclassified']:
            return 2
    if (speed > 35 and lanes <= 2) or (not bike and lanes == 3):
        if fc in ['tertiary', 'secondary']:
            return 3
    if speed > 50 or lanes >= 4 or (not bike and fc in ['primary', 'trunk', 'motorway']):
        return 4
    return 3

#执行 LTS 分类
edges['LTS'] = edges.apply(assign_lts, axis=1)

In [ ]:
# 确保 LTS 字段为整数（防止 float 比较错误）
edges['LTS'] = edges['LTS'].astype('Int64')

# 绘图：不同 LTS 等级上色
lts_colors = {1: '#2ca02c', 2: '#1f77b4', 3: '#ff7f0e', 4: '#d62728'}

fig, ax = plt.subplots(figsize=(14, 14))
for level, color in lts_colors.items():
    subset = edges[edges['LTS'] == level]
    if not subset.empty:
        subset.plot(ax=ax, color=color, linewidth=0.5, label=f'LTS {level}')

ax.legend(title='LTS Level')
ax.set_title('Greater London – LTS Road Classification')
ax.set_axis_off()
plt.tight_layout()
plt.show()